## LangChain — Chat Models: Tool Calling with Ollama (DeepSeek R1 8B)

This notebook demonstrates how to use chat models to call tools using LangChain's tool-calling interface with `ChatOllama` and the `deepseek-r1:8b` model running locally via Ollama.

References:
- LangChain — Tool calling: https://python.langchain.com/docs/how_to/tool_calling/
- Notebook reference: https://raw.githubusercontent.com/langchain-ai/langchain/refs/heads/master/docs/docs/how_to/tool_calling.ipynb

Assumptions:
- Ollama is running locally at `http://localhost:11434`
- The model `deepseek-r1:8b` is installed (run `ollama pull deepseek-r1:8b` if needed)

What you'll learn:
- Define tools with `@tool`
- Bind tools to a chat model (`bind_tools`)
- Inspect `tool_calls` emitted by the model
- Execute tools and feed results back to the model to complete the answer



### 1) Install and verify environment

Use the cell below to install the required packages. You can skip installation if you already have them.


In [24]:
!python -V
%pip install -qU langchain langchain-core langchain-ollama pydantic rich
# If not present, pull the model used for a basic check:
# ollama pull deepseek-r1:8b
# Also pull a tool-capable model used in this notebook (any of these is fine):
# ollama pull qwen2.5:7b-instruct
# ollama pull llama3.1:8b-instruct-q8_0


Python 3.12.11

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 2) Connect to Ollama and sanity check

We'll connect to the local Ollama server via `ChatOllama` and ensure the model responds.


In [25]:
from langchain_ollama import ChatOllama

# Basic connectivity check to Ollama
# model = "deepseek-r1:8b"
model = "llama3.1:8b-instruct-q8_0"
llm = ChatOllama(model=model, temperature=0)
resp = llm.invoke("Return the word 'ready'.")
print(resp.content)


The word is: ready.


### 3) Define tools with `@tool`

We'll create simple tools and annotate them so the model understands what they do and the arguments they accept.


In [26]:
from typing import Literal
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two integers and return the sum.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the product.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

@tool
def convert_temperature(value: float, unit: Literal["C_to_F", "F_to_C"]) -> float:
    """Convert temperatures between Celsius and Fahrenheit.

    - Use unit="C_to_F" to convert Celsius to Fahrenheit
    - Use unit="F_to_C" to convert Fahrenheit to Celsius
    """
    if unit == "C_to_F":
        return (value * 9 / 5) + 32
    else:
        return (value - 32) * 5 / 9

TOOLS = [add, multiply, convert_temperature]


### 4) LangChain Tool

In addition to the `@tool` decorator on functions, you can build a LangChain Tool object. `StructuredTool` wraps a Python function with an auto-generated JSON schema so models can call it.


In [27]:
from langchain_core.tools import StructuredTool


def power(base: int, exponent: int) -> int:
    return base ** exponent

power_tool = StructuredTool.from_function(
    func=power,
    name="power",
    description="Raise a base to an exponent and return the integer result.",
)

TOOLS_WITH_STRUCTURED = TOOLS + [power_tool]



### 5) Pydantic class

You can define tool schemas using `pydantic` models. The model serves as the tool schema and provides validation and rich descriptions for arguments.


In [28]:
from pydantic import BaseModel, Field
import json
from urllib.request import urlopen, Request

# Helper: pick a tool-capable model that's already installed in Ollama
PREFERRED_TOOL_MODELS = [
    "llama3.1:8b-instruct-q8_0",
    "qwen2.5:7b-instruct",
    "deepseek-r1:8b",
]

def list_ollama_models() -> list[str]:
    try:
        req = Request("http://localhost:11434/api/tags")
        with urlopen(req, timeout=2) as resp:
            data = json.loads(resp.read().decode("utf-8"))
            return [m.get("name", "") for m in data.get("models", [])]
    except Exception:
        return []

def get_tool_capable_llm() -> ChatOllama:
    available_models = list_ollama_models()
    selected = next((m for m in PREFERRED_TOOL_MODELS if m in available_models), None)
    if selected is None:
        raise RuntimeError(
            "No tool-capable model found in Ollama. Please run: `ollama pull qwen2.5:7b-instruct`"
        )
    return ChatOllama(model=selected, temperature=0)

# Use the selected tool-capable model for tool calling demos
llm_tools = get_tool_capable_llm()


class TemperatureConversion(BaseModel):
    """Convert temperatures between Celsius and Fahrenheit."""

    value: float = Field(..., description="Temperature value to convert")
    unit: Literal["C_to_F", "F_to_C"] = Field(
        ..., description="Conversion direction: 'C_to_F' or 'F_to_C'"
    )

    def execute(self) -> float:
        if self.unit == "C_to_F":
            return (self.value * 9 / 5) + 32
        return (self.value - 32) * 5 / 9


# Bind a model with a Pydantic tool schema
llm_with_pydantic = llm_tools.bind_tools([TemperatureConversion])



### 6) TypedDict class

Alternatively, you can define tool schemas using `TypedDict` for lightweight, typed dictionaries.


In [29]:
from typing import TypedDict
from pydantic import create_model


class AddArgs(TypedDict):
    a: int
    b: int

# Convert TypedDict to a Pydantic model for args_schema compatibility
AddArgsModel = create_model("AddArgsModel", a=(int, ...), b=(int, ...))


@tool(args_schema=AddArgsModel)
def add_typed(a: int, b: int) -> int:
    """Add two integers and return the sum.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


llm_with_typeddict = llm_tools.bind_tools([add_typed])



### 7) Tool calls

Let's see the model propose tool calls and how to view them. We'll ask a multi-part math question that should trigger multiple tools.


In [30]:
# Bind the function tools and the structured tool together (use tool-capable model)
llm_with_tools = llm_tools.bind_tools(TOOLS_WITH_STRUCTURED)

query = "What is 3 * 12? Also, what is 11 + 49? And compute 2^10."
msg = llm_with_tools.invoke(query)

print("Model content:\n", msg.content)
print("\nTool calls:")
for i, tc in enumerate(getattr(msg, "tool_calls", []) or []):
    print(i, tc["name"], tc["args"])


Model content:
 

Tool calls:
0 multiply {'a': 3, 'b': 12}
1 add {'a': 11, 'b': 49}
2 power {'base': 2, 'exponent': 10}


### 8) Parsing

We can parse tool calls into structured Python objects for easier execution and testing. Below demonstrates parsers for function-based tools, Pydantic tool schemas, and a JSON parser for responses.


In [ ]:
from langchain_core.output_parsers import (
    JsonOutputToolsParser,
    PydanticToolsParser,
    JsonOutputParser,
)

# 8.1) Parse Pydantic-modeled tool calls into Pydantic objects
pydantic_parser = PydanticToolsParser(tools=[TemperatureConversion])
msg_pyd = llm_with_pydantic.invoke("Convert 20 C to Fahrenheit.")
parsed_pyd = pydantic_parser.invoke(msg_pyd)
print("Parsed pydantic tool calls:")
for obj in parsed_pyd:
    print(type(obj).__name__, obj)

# 8.2) Parse plain JSON text with JsonOutputParser
json_text = '{"tools": [{"type": "add", "args": {"a": 11, "b": 49}}]}'
plain_json_parser = JsonOutputParser()
print("\nParsed JSON text:", plain_json_parser.invoke(json_text))

# 8.3) Parse tool calls from a chat message using JsonOutputToolsParser (uses `msg` from above)
tools_parser = JsonOutputToolsParser()
print("\nParsed tool calls from AIMessage:", tools_parser.invoke(msg))



Parsed pydantic tool calls:
TemperatureConversion value=20.0 unit='C_to_F'


OutputParserException: This output parser can only be used with a chat generation.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 